In [1]:
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

In [2]:
train_image_directory = "./bttai-nybg-2024/BTTAIxNYBG-train/BTTAIxNYBG-train"
validation_image_directory = "./bttai-nybg-2024/BTTAIxNYBG-validation/BTTAIxNYBG-validation"


In [17]:
filename_train = "./bttai-nybg-2024/BTTAIxNYBG-train.csv"
df_train = pd.read_csv(filename_train, header = 0)
filename_test = "./bttai-nybg-2024/BTTAIxNYBG-test.csv"
df_test = pd.read_csv(filename_test, header = 0)
filename_val = "./bttai-nybg-2024/BTTAIxNYBG-validation.csv"
df_val = pd.read_csv(filename_val, header = 0)

In [3]:
base_model = InceptionV3(weights='imagenet', include_top=False)



87910968/87910968 [==============================] - 2s 0us/step


In [4]:
model = Model(inputs=base_model.input, outputs=base_model.output)
# we are only interested in the features, not the classification

In [6]:
datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [18]:
train_generator = datagen.flow_from_dataframe(
    dataframe=df_train,
    directory=train_image_directory,
    x_col='imageFile',
    y_col='classLabel',
    target_size=(299, 299),  # Size expected by InceptionV3
    batch_size=32,
    class_mode='raw',  # Since we're only interested in features, not class labels
    shuffle=False)  # Keep data in same order to match labels

Found 81946 validated image filenames.


In [19]:
def extract_features(generator, model):
    features = model.predict_generator(generator, steps=len(generator), verbose=1)
    # also look how I am using predict
    # flatten the features to make them suitable for traditional classifiers
    features_flattened = features.reshape(features.shape[0], -1)
    return features_flattened

In [20]:
train_features = extract_features(train_generator, model)

C:\Users\rhasa\AppData\Local\Temp\ipykernel_20208\938002562.py:2: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  features = model.predict_generator(generator, steps=len(generator), verbose=1)


   6/2561 [..............................] - ETA: 50:36

KeyboardInterrupt: 

In [ ]:
train_labels = train_generator.classes
# np.save('train_features.npy', train_features)
# np.save('train_labels.npy', train_labels)